In [ ]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import pare, plurality_env, Roles, Phase
from notebooks.learning_agents.models import ActorCriticAgent
from notebooks.learning_agents.utils import play_static_game, play_recurrent_game
from notebooks.learning_agents.static_agents import (
    random_plurality_wolf, 
    random_approval_wolf,
    )
import notebooks.learning_agents.stats as indicators
import random
import copy
from matplotlib import pyplot as plt
from tqdm import tqdm
from tabulate import tabulate

# Discussion


## Training 
### Inconsistent Training

Reinforcement learning is a hard problem, and MARL even more so given its dynamic nature. To train our agents, we use a policy gradient method (PPO) that consistently improves our agents, but almost always has some type of divergence or weight collapse. In Sutton and Barto **REF**, they refer to a combination of issues -**deadly triad**- that when used together, lead to divergence more often than not. These issues are :
- Using non-linear function approximations. _Our neural network_
- Bootstraping our estimated values. _Our policy is changing constantly, along with our value approximation_
- Off-Policy Training. _The one thing we do not do in this case!_

Another thing to note is the difficulty to calcuate good estimates of the policy gradient (cite Kakade Langfor 2002) given the stochasticity of our MARL environment, and if we get a couple unfortunate episodes with bad estiamtes, our parameters may change in a poor direction leading to policy collapse and a long to possibly never recovery time. We do see in our training that there are multiple such events, with and without future recovery.

Empirically, our agents can train for around 500 update steps around $10\%$ of the time, meaning a collapse of our weights is almost always the termination factor of training. Despite this we do get decent results in the meantime, and would like to make training more consistent and reduce this collapse.

Some ideas we believe may help and are worth trying:
- Using decaying rates for our learning rate
- Use running norms for observations and rewards
- futher explore gradient clipping
- any divisions should also add an $\epsilon$ of $1e-5$ to begin with
- clamp logarithmic values between `-np.log(1e-5) , np.log(1e-5))` to begin with
- split critic and actor networks
    - have a higher learning rate for the critic
- vary replay buffer sizes
- vary batch sizes
- change optimizer
- simplify model strcuture

These have been collected from many blog posts, reddit posts, and work done in {cite}`Andrychowicz2020-fs`. We leave this exploration to future work, as it is not the direct scope of the project, but would help with consistency.


### Training Times

One game of plurality takes :
One game of approval takes :

We 

## Approval vs Plurality

Approval Voting allowed for agents to express dislikes as well as likes against all other agents in the game. This increased expressibility led to harder to interpret behavior, although trends were clearly seen. Training approval agents was more consistent than plurality agents, and they achieved higher win-rates much quicker. 

